In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

'''
borrowed from https://blog.csdn.net/fengdu78/article/details/104337519
'''

class mynet(nn.Module):
    def __init__(self, n_components = 5, inner_size = 16, act_layer = nn.ReLU()):
        super(mynet, self).__init__()
        self.n_components = n_components
        self.inner_size = inner_size
        self.net = nn.Sequential(
            # [b, 41] => [b, rank]
            nn.Linear(41, 16),
            nn.Sigmoid(),
            nn.Linear(16, self.n_components),
            nn.ReLU(),
            # [b, rank] => [b, 41]
            nn.Linear(self.n_components, 16),
            nn.Sigmoid(),
            nn.Linear(16, 41),
            nn.ReLU(),
        )
 
 
    def forward(self, x):
        """
        :param [b, 1, 28, 28]:
        :return [b, 1, 28, 28]:
        """
        batchsize = x.size(0)
        # flatten
        x = x.view(batchsize, -1)
        # encoder
        x = self.net(x)
        # reshape
        x = x.view(batchsize, -1)
 
        return x

    def encode(self, x):
        # batchsize = x.size(0)
        # return self.encoder(x.view(batchsize,-1)).view(batchsize,-1)
        return self.encoder(x)

def fit(x, y, n_components = 5):
    epochs = 10
    lr = 1e-3
    model = net(n_components)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    print(model)

    tensor_x=torch.from_numpy(x.astype(np.float32))
    tensor_y=torch.from_numpy(y.astype(np.float32))
    my_dataset=TensorDataset(tensor_x,tensor_y)
    my_dataset_loader=DataLoader(my_dataset,batch_size=2000,shuffle=False)

    for epoch in range(epochs):
        print(epoch)
        total_loss = 0
        for batchidx, (x, y) in enumerate(my_dataset_loader):
            pred = model(x)
            loss=criterion(pred,x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss
        # if epoch % 100==0:

        print(total_loss.data.numpy())
    
    return model

ae = fit(train_features_raw.to_numpy(), train_labels.to_numpy())
train_features = ae.encode(train_features_raw.to_numpy())
test_features = ae.encode(test_features_raw.to_numpy())
train_features

precision, recall, F1_score, _ = precision_recall_fscore_support(test_labels, y_pred, average=None)
acc = accuracy_score(test_labels, y_pred)
print("precision: {} \nrecall: {} \nF1 score: {} \naccuracy: {}".format(precision, recall, F1_score, acc))